In [7]:
pip install lunarcalendar


--- Logging error ---
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/Library/

In [10]:
from datetime import datetime
from lunarcalendar import Converter, Solar

def get_ganzhi(year, month, day, hour):
    # 定义干支表
    heavenly_stems = ["甲", "乙", "丙", "丁", "戊", "己", "庚", "辛", "壬", "癸"]
    earthly_branches = ["子", "丑", "寅", "卯", "辰", "巳", "午", "未", "申", "酉", "戌", "亥"]
    
    # 公历转农历
    solar_date = Solar(year, month, day)
    lunar_date = Converter.Solar2Lunar(solar_date)
    
    # 获取农历日索引
    lunar_day_index = lunar_date.day  # 农历日直接取其值
    
    # 计算日干支
    base_date = datetime(1900, 1, 31)  # 甲辰年正月初一
    current_date = datetime(year, month, day)
    delta_days = (current_date - base_date).days
    day_gan_index = (delta_days + 3) % 10
    day_zhi_index = (delta_days + 3) % 12
    day_ganzhi = heavenly_stems[day_gan_index] + earthly_branches[day_zhi_index]
    
    # 计算时干支
    hour_zhi_index = (hour + 1) // 2 % 12
    hour_gan_index = (day_gan_index * 2 + hour_zhi_index) % 10
    hour_ganzhi = heavenly_stems[hour_gan_index] + earthly_branches[hour_zhi_index]
    hour_branch = earthly_branches[hour_zhi_index]  # 获取时支单独值
    
    return {
        "solar_date": f"{year}-{month}-{day}",
        "lunar_date": f"农历{lunar_date.year}年{lunar_date.month}月{lunar_date.day}日",
        "lunar_day_index": lunar_day_index,
        "day_ganzhi": day_ganzhi,
        "hour_ganzhi": hour_ganzhi,
        "hour_branch": hour_branch
    }




In [16]:
def xiaoliuren_pan(lunar_day_index, hour_branch, day_branch):
    # 定义规则表
    six_positions = ["大安", "留连", "速喜", "赤口", "小吉", "空亡"]
    six_gods = ["青龙", "朱雀", "勾陈", "白虎", "玄武", "腾蛇"]
    five_elements = ["木", "火", "土", "金", "水"]
    branch_order = ["子", "丑", "寅", "卯", "辰", "巳", "午", "未", "申", "酉", "戌", "亥"]
    branch_elements = {
        "子": "水", "丑": "土", "寅": "木", "卯": "木",
        "辰": "土", "巳": "火", "午": "火", "未": "土",
        "申": "金", "酉": "金", "戌": "土", "亥": "水"
    }
    element_cycle = ["木", "火", "土", "金", "水"]  # 五行生克顺序

    # 找到初始宫位
    day_position_index = (lunar_day_index - 1) % 6
    day_position = six_positions[day_position_index]
    
    # 时支宫位
    hour_position_index = (day_position_index + branch_order.index(hour_branch)) % 6
    hour_position = six_positions[hour_position_index]
    
    # 分配时支到各宫
    distributed_branches = {}
    current_index = hour_position_index
    for i in range(6):
        distributed_branches[six_positions[current_index]] = branch_order[(branch_order.index(hour_branch) + 2 * i) % 12]
        current_index = (current_index + 1) % 6

    # 排六神
    god_start_index = (branch_order.index(hour_branch)) % 6
    gods = {six_positions[(god_start_index + i) % 6]: six_gods[i] for i in range(6)}

    # 排五星
    element_start_index = day_position_index
    elements = {six_positions[(element_start_index + i) % 6]: five_elements[i] for i in range(5)}

    # 确定自身
    hour_element = branch_elements[hour_branch]  # 自身五行
    relatives = {}
    for position, branch in distributed_branches.items():
        branch_element = branch_elements[branch]
        if branch == hour_branch:
            relatives[position] = "自身"

    # 计算其余六亲
    for position, branch in distributed_branches.items():
        branch_element = branch_elements[branch]
        if position == hour_position:
            relatives[position] = "自身"
        elif branch_element == element_cycle[(element_cycle.index(hour_element) - 2) % 5]:
            relatives[position] = "官鬼"  # 克自身
        elif branch_element == element_cycle[(element_cycle.index(hour_element) + 2) % 5]:
            relatives[position] = "妻财"  # 自身克
        elif branch_element == element_cycle[(element_cycle.index(hour_element) - 1) % 5]:
            relatives[position] = "父母"  # 生自身
        elif branch_element == element_cycle[(element_cycle.index(hour_element) + 1) % 5]:
            relatives[position] = "子孙"  # 被自身生
        else:
            relatives[position] = "兄弟"  # 默认值
    
    if hour_branch in ["寅", "申", "巳", "亥"]:
        # 自身开始数第二个为兄弟
        siblings_index = (list(distributed_branches.keys()).index(hour_position) + 1) % 6
        siblings_position = list(distributed_branches.keys())[siblings_index]
        relatives[siblings_position] = "兄弟"
    elif hour_branch in ["子", "午", "卯", "酉"]:
        # 自身开始数第三个为兄弟
        siblings_index = (list(distributed_branches.keys()).index(hour_position) + 2) % 6
        siblings_position = list(distributed_branches.keys())[siblings_index]
        relatives[siblings_position] = "兄弟"

    # 输出结果
    result = []
    for position in six_positions:
        result.append(f"{position}宫：{distributed_branches.get(position, '无')} "
                      f"六神：{gods.get(position, '无')} "
                      f"五星：{elements.get(position, '无')} "
                      f"六亲：{relatives.get(position, '无')}")
    return "\n".join(result)


# 示例调用程序
if __name__ == "__main__":
    # 输入年月日时
    year = int(input("请输入年份（如2024）："))
    month = int(input("请输入月份（如11）："))
    day = int(input("请输入日期（如23）："))
    hour = int(input("请输入时辰（如14）："))
    
    # 计算干支
    ganzhi_result = get_ganzhi(year, month, day, hour)
    lunar_day_index = ganzhi_result['lunar_day_index']
    day_ganzhi = ganzhi_result['day_ganzhi']
    hour_ganzhi = ganzhi_result['hour_ganzhi']
    hour_branch = ganzhi_result['hour_branch']
    print(f"日干支：{day_ganzhi}，时干支：{hour_ganzhi}")
    
    # 四盘小六壬
    result = xiaoliuren_pan(lunar_day_index, hour_branch, day_ganzhi)
    print("四盘小六壬结果：")
    print(result)


日干支：癸丑，时干支：己未
四盘小六壬结果：
大安宫：亥 六神：腾蛇 五星：金 六亲：妻财
留连宫：丑 六神：青龙 五星：水 六亲：兄弟
速喜宫：卯 六神：朱雀 五星：无 六亲：官鬼
赤口宫：巳 六神：勾陈 五星：木 六亲：父母
小吉宫：未 六神：白虎 五星：火 六亲：自身
空亡宫：酉 六神：玄武 五星：土 六亲：子孙


In [12]:
def interpret_xiaoliuren_results(results, question_type):
    """
    根据小六壬的结果和问题类型进行解读。
    :param results: 小六壬的结果字符串，每个宫的信息应包含六神、六亲、五星、宫位等。
    :param question_type: 问题类型，1-运势，2-感情，3-工作，4-学业。
    :return: 解析结果的字符串。
    """
    interpretations = []

    # 定义问题类型
    question_types = {
        1: "运势",
        2: "感情",
        3: "工作",
        4: "学业"
    }
    question_label = question_types.get(question_type, "未知问题类型")

    # 六神解析规则
    god_meanings = {
        "青龙": "吉利，代表好运、贵人。",
        "朱雀": "口舌是非，沟通、争执相关。",
        "勾陈": "拖延、反复，可能受阻。",
        "白虎": "凶险，意外、竞争或压力。",
        "玄武": "隐秘、欺骗，注意虚实。",
        "腾蛇": "虚幻，迷惑，灵感或意外情况。"
    }

    # 五星解析规则
    element_meanings = {
        "木": "生机、成长，适合计划和新开始。",
        "火": "热情、行动，适合快速推进。",
        "土": "稳定、积累，适合长期规划。",
        "金": "决断、收获，适合总结和结果。",
        "水": "流动、变化，适合学习和灵活调整。",
        "无": "不确定的能量，需要谨慎观察。"
    }

    # 遍历每一行结果
    for result in results.strip().split("\n"):
        if not result.strip():
            continue  # 跳过空行

        try:
            # 使用空格分隔结果部分
            parts = result.split()
            position = parts[0].split("宫：")[0]  # 宫位
            branch = parts[0].split("：")[1]  # 地支
            god = parts[1].split("：")[1]  # 六神
            element = parts[2].split("：")[1]  # 五星
            relative = parts[3].split("：")[1]  # 六亲
        except (IndexError, ValueError):
            # 如果分割失败，标记为解析失败
            interpretations.append(f"解析失败：{result}")
            continue

        # 根据问题类型生成解析
        if question_type == 1:  # 运势
            if relative == "自身":
                analysis = "运势平稳，适合按计划推进。"
            elif relative == "官鬼":
                analysis = "运势受阻，需警惕外界压力或竞争。"
            elif relative == "妻财":
                analysis = "运势顺畅，资源充足，利于投资或收益。"
            elif relative == "父母":
                analysis = "运势良好，易得贵人相助，适合求援。"
            elif relative == "子孙":
                analysis = "运势积极，有新机会出现，适合创新。"
            elif relative == "兄弟":
                analysis = "运势波动，可能有内部竞争或合作机会。"
            else:
                analysis = "运势一般，无大起大落。"

        elif question_type == 2:  # 感情
            if relative == "自身":
                analysis = "感情现状平稳，需关注对方需求，维系感情稳定。"
            elif relative == "官鬼":
                analysis = "感情中可能有阻碍或矛盾，需加强沟通化解。"
            elif relative == "妻财":
                analysis = "感情甜蜜，有望进一步发展，适合表白或深入沟通。"
            elif relative == "父母":
                analysis = "感情发展顺利，家人或朋友对关系表示支持。"
            elif relative == "子孙":
                analysis = "感情中可能有新的机会或新对象出现。"
            elif relative == "兄弟":
                analysis = "感情可能受外界因素干扰或有竞争关系。"
            else:
                analysis = "感情平淡，需注入更多新鲜感。"

        elif question_type == 3:  # 工作
            if relative == "自身":
                analysis = "工作状态稳定，适合稳步推进当前任务。"
            elif relative == "官鬼":
                analysis = "工作压力较大，可能遇到挑战或竞争对手。"
            elif relative == "妻财":
                analysis = "工作顺利，适合争取资源或申请加薪。"
            elif relative == "父母":
                analysis = "工作中易得上级或长辈帮助，适合请教经验。"
            elif relative == "子孙":
                analysis = "工作中有新的机会，适合尝试创新或新项目。"
            elif relative == "兄弟":
                analysis = "工作中可能有团队竞争或协作问题。"
            else:
                analysis = "工作状态一般，需稳中求进。"

        elif question_type == 4:  # 学业
            if relative == "自身":
                analysis = "学业状态良好，适合集中精力学习。"
            elif relative == "官鬼":
                analysis = "学习上有压力或困难，需要调整学习计划。"
            elif relative == "妻财":
                analysis = "学业上可能获得奖励或成果，适合考试。"
            elif relative == "父母":
                analysis = "学业中易得老师或长辈支持，适合寻求指导。"
            elif relative == "子孙":
                analysis = "学习状态积极，有机会接触新领域或新知识。"
            elif relative == "兄弟":
                analysis = "学习中可能与同学产生竞争或合作关系。"
            else:
                analysis = "学业状态平稳，无明显突破。"

        else:
            analysis = "暂无解析规则，请扩展功能。"

        # 综合六神和五星的含义
        god_meaning = god_meanings.get(god, "未知")
        element_meaning = element_meanings.get(element, "未知")
        combined_analysis = f"六神“{god}”寓意{god_meaning}，五星“{element}”表示{element_meaning}。"

        # 最终解释
        interpretations.append(
            f"“{god}”临“{relative}”落“{position}”，五星为“{element}”。解析：{analysis} {combined_analysis}"
        )

    return f"问题类型：{question_types.get(question_type, '未知')}\n" + "\n".join(interpretations)


In [54]:
def interpret_six_god_with_palace(palace, god, question_type):
    """
    根据问题类型，结合宫位和六神的意义进行动态解释。
    :param palace: 宫位名称。
    :param god: 六神名称。
    :param question_type: 问题类型，1-运势，2-感情，3-工作，4-学业，5-占财，6-身体状况。
    :return: 针对问题类型的解释。
    """
    meanings_by_question = {
        1: {  # 运势
            "青龙": {
                "大安": "贵人助力，事事顺利，整体运势佳。",
                "留连": "虽有拖延，但好运临近，适合等待。",
                "速喜": "运势旺盛，快速成功，需果断行动。",
                "赤口": "需警惕小人，避免过度激进。",
                "小吉": "小有成功，值得继续推进。",
                "空亡": "运势受阻，宜另寻出路。"
            },
            "朱雀": {
                "大安": "谨慎言行，防止因言语过多引发问题。",
                "留连": "进展缓慢，易因沟通问题受阻。",
                "速喜": "快速推进中注意沟通问题。",
                "赤口": "易因争执导致进展受阻。",
                "小吉": "可推进小事务，但需多沟通。",
                "空亡": "运势迷茫，需耐心等待机会。"
            },
            "勾陈": {
                "大安": "运势平稳，但进展可能略显拖延。",
                "留连": "事情受阻，可能需要重复努力。",
                "速喜": "行动快速但可能带来反复。",
                "赤口": "运势受阻，易因冲突产生拖延。",
                "小吉": "稳定推进，但可能小有波折。",
                "空亡": "停滞不前，宜重新审视方向。"
            },
            "白虎": {
                "大安": "平稳中需提防风险或隐患。",
                "留连": "进展缓慢，易出现意外状况。",
                "速喜": "快速推进但可能伴随风险。",
                "赤口": "争执或压力较多，需谨慎行事。",
                "小吉": "小有所成，但需防范潜在问题。",
                "空亡": "进展不顺，宜规避冒险行为。"
            },
            "玄武": {
                "大安": "表面平稳但暗藏隐患，需注意细节。",
                "留连": "事情进展缓慢，可能隐藏机会。",
                "速喜": "快速推进但注意隐藏因素的影响。",
                "赤口": "冲突中隐藏隐秘问题，需冷静应对。",
                "小吉": "有所收获，但仍需关注隐患。",
                "空亡": "迷茫状态中，需仔细观察隐藏的机会。"
            },
            "腾蛇": {
                "大安": "运势表象良好，但可能有虚幻成分。",
                "留连": "易受迷惑，需谨慎决策。",
                "速喜": "快速行动中需防范虚假机会。",
                "赤口": "表象迷惑，需冷静分析后再行动。",
                "小吉": "小有收获，但可能伴随虚幻成分。",
                "空亡": "停滞不前，但可能会获得意外启示。"
            }
        },
        2: {  # 感情
            "青龙": {
                "大安": "感情稳定，彼此信任，关系持续发展。",
                "留连": "感情稍有停滞，但前景向好。",
                "速喜": "关系快速发展，适合推进。",
                "赤口": "情感矛盾增多，需谨慎应对。",
                "小吉": "感情略有进展，适合维系现状。",
                "空亡": "感情模糊，可能难以推进。"
            },
            "朱雀": {
                "大安": "平稳中易因言辞导致误会。",
                "留连": "沟通不畅，感情发展缓慢。",
                "速喜": "快速发展中需注意沟通问题。",
                "赤口": "易起争执，需注意情绪控制。",
                "小吉": "感情小有进展，但需加强沟通。",
                "空亡": "感情现状不明，需观察耐心等待。"
            },
            "勾陈": {
                "大安": "感情稳定但可能进展缓慢。",
                "留连": "感情发展停滞，需更多耐心。",
                "速喜": "有感情突破，但易有反复。",
                "赤口": "感情受阻，需避免矛盾激化。",
                "小吉": "小有感情进展，值得继续维系。",
                "空亡": "感情状态不明朗，需等待机会。"
            },
            "白虎": {
                "大安": "感情稳定中需提防潜在风险。",
                "留连": "感情停滞，易受外部因素干扰。",
                "速喜": "快速发展但可能带来矛盾。",
                "赤口": "争执或分歧较多，需谨慎处理。",
                "小吉": "感情略有改善，但需防范风险。",
                "空亡": "感情停滞，难有进展，需另寻机会。"
            },
            "玄武": {
                "大安": "表面和谐，可能隐藏问题，需关注细节。",
                "留连": "感情发展缓慢，可能有隐秘障碍。",
                "速喜": "感情迅速发展，但隐藏潜在问题。",
                "赤口": "情感矛盾中可能有隐秘因素。",
                "小吉": "感情略有进展，但需防范隐患。",
                "空亡": "感情状态迷茫，需冷静分析形势。"
            },
            "腾蛇": {
                "大安": "感情表面和谐，但可能存在虚幻成分。",
                "留连": "感情发展中可能易受迷惑。",
                "速喜": "短期关系进展，但可能缺乏稳定性。",
                "赤口": "情感矛盾中可能伴随误导因素。",
                "小吉": "感情略有改善，但可能存在虚幻。",
                "空亡": "感情进展困难，但可能有意外启示。"
            }
        },
        3: {  # 工作
             "青龙": {
                 "大安": "工作稳定，贵人相助，适合长期发展。",
                 "留连": "工作略有拖延，但前景较好。",
                 "速喜": "快速晋升或机会出现，适合主动争取。",
                 "赤口": "职场矛盾多，需避免争执与竞争。",
                 "小吉": "工作略有进展，适合稳中求进。",
                 "空亡": "工作停滞，需重新规划方向。"
            },
             "朱雀": {
                  "大安": "需注意沟通问题，避免因言语失误引发问题。",
                 "留连": "工作进展缓慢，沟通中易生误解。",
                 "速喜": "短期内有突破，但沟通不畅可能带来问题。",
                 "赤口": "争执与压力增加，职场需冷静应对。",
                 "小吉": "工作略有进展，适合维持现状。",
                 "空亡": "工作方向不明，建议等待时机再行动。"
            },
             "勾陈": {
                 "大安": "工作平稳，但进展可能较为缓慢。",
                 "留连": "工作易受阻力，需耐心解决问题。",
                 "速喜": "短期内可能快速推进，但后续需反复调整。",
                 "赤口": "工作矛盾多，可能因冲突导致拖延。",
                 "小吉": "工作稳定但进展不快，需保持耐心。",
                 "空亡": "工作停滞不前，宜重新审视规划。"
            },
             "白虎": {
                 "大安": "工作较为稳定，但需防范潜在风险。",
                 "留连": "拖延中可能出现意外问题，宜多加注意。",
                 "速喜": "快速推进中伴随一定风险，需谨慎。",
                 "赤口": "压力较大，职场矛盾多，需冷静处理。",
                 "小吉": "工作略有进展，但需警惕潜在隐患。",
                 "空亡": "工作受阻，难以推进，需调整方向。"
            },
             "玄武": {
                 "大安": "表面稳定，但可能存在隐性问题或欺骗。",
                 "留连": "工作进展缓慢，隐藏的障碍需解决。",
                 "速喜": "短期内有突破，但需防范隐性问题。",
                 "赤口": "工作矛盾中隐藏隐患，需冷静处理。",
                 "小吉": "有所收获，但仍需关注潜在风险。",
                 "空亡": "工作状况不明朗，可能需要重新评估。"
            },
             "腾蛇": {
                 "大安": "工作表面无碍，但可能存在虚幻或误导。",
                 "留连": "拖延中易被迷惑，需警惕不确定因素。",
                 "速喜": "快速突破中可能伴随虚假机会。",
                 "赤口": "矛盾增多，可能因虚假信息受阻。",
                 "小吉": "工作有所进展，但需警惕虚假承诺。",
                 "空亡": "工作停滞不前，但可能存在意外转机。"
            }
        },
        4: {  
             "青龙": {
                 "大安":   "学习状态稳定，适合积累基础知识。",
                 "留连": "学习进展缓慢，但潜力较大。",
                 "速喜": "考试或学习成果快速显现，适合集中突破。",
                 "赤口": "注意因急躁导致的失误，需冷静复习。",
                 "小吉": "学业小有进步，适合持续努力。",
                 "空亡": "学习方向迷茫，需调整计划。"
            },
             "朱雀": {
                 "大安": "学习需谨慎，防止因粗心影响成绩。",
                 "留连": "学习中易有反复，需解决理解问题。",
                 "速喜": "短期内学习成果明显，但易因争议受阻。",
                 "赤口": "学习状态不佳，可能因外界干扰受挫。",
                 "小吉": "学习略有提升，需注意复习巩固。",
                 "空亡": "学习无突破，需调整节奏。"
            },
             "勾陈": {
                 "大安": "学习稳定，但进展缓慢。",
                 "留连": "学习容易受阻，需耐心解决问题。",
                 "速喜": "快速提升但可能面临反复。",
                 "赤口": "学习冲突多，可能因矛盾受阻。",
                 "小吉": "学习略有进步，但需警惕潜在问题。",
                 "空亡": "学习停滞不前，需调整计划。"
            },
             "白虎": {
                 "大安": "学习状态较稳定，但需注意压力。",
                 "留连": "进展缓慢，可能伴随不良影响。",
                 "速喜": "快速学习但可能伴随一定风险。",
                 "赤口": "冲突或压力增多，需加强调整。",
                 "小吉": "小有提升，但需注意过度疲劳。",
                 "空亡": "学习陷入低谷，建议重新调整方向。"
            },
              "玄武": {
                 "大安": "学习状态平稳，但可能隐藏问题。",
                 "留连": "学习进展较慢，需防范隐性问题。",
                 "速喜": "学习成果显现，但需关注潜在障碍。",
                 "赤口": "学习冲突中可能有隐性问题，需解决。",
                 "小吉": "学习有所突破，但需避免隐患影响。",
                 "空亡": "学习方向迷茫，可能需外界指导。"
            },
             "腾蛇": {
                 "大安": "表面学习无碍，但可能存在误导或偏差。",
                 "留连": "学习拖延中可能易受误导。",
                 "速喜": "短期内学习成果显现，但可能有偏差。",
                 "赤口": "冲突增加，需避免情绪干扰学习。",
                 "小吉": "学习有进步，但需警惕虚假提升。",
                 "空亡": "学习状态低迷，但可能存在意外启示。"
            }
        },
        5: {  # 占财
             "青龙": {
                 "大安": "财运稳定，贵人相助，适合长期投资。",
                 "留连": "财运略有拖延，但前景较好。",
                 "速喜": "短期内财运快速提升，需抓住机会。",
                 "赤口": "财务矛盾多，需谨慎处理争执。",
                 "小吉": "财运小有进账，适合积累。",
                 "空亡": "财运较差，暂不宜进行大额投入。"
            },
             "朱雀": {
                 "大安": "需注意沟通问题，防止因言语失误引发财务问题。",
                 "留连": "财运受阻，可能因沟通问题延迟收益。",
                 "速喜": "短期内收益可见，但需注意财务沟通问题。",
                 "赤口": "财务纠纷易起，需冷静处理避免损失。",
                 "小吉": "小有财运进账，但需注意资金分配。",
                 "空亡": "财运停滞，需等待时机，不宜冒进。"
            },
             "勾陈": {
                 "大安": "财务平稳，但收益可能较慢。",
                 "留连": "财运受阻，可能有反复波折，宜耐心。",
                 "速喜": "短期内收益可能有所增加，但需反复核实。",
                 "赤口": "财务争执或拖延，需谨慎处理合同与协议。",
                 "小吉": "财运小幅提升，但可能存在反复。",
                 "空亡": "财运停滞不前，建议重新调整财务规划。"
            },
             "白虎": {
                 "大安": "财运稳定，但需警惕潜在风险。",
                 "留连": "拖延中可能出现意外损失，宜多加注意。",
                 "速喜": "快速收益中伴随风险，需谨慎投资。",
                 "赤口": "压力较大，可能因矛盾或竞争导致财务损失。",
                 "小吉": "财运有小幅提升，但需警惕突发事件。",
                 "空亡": "财务状况不佳，可能有较大风险，宜保守。"
            },
             "玄武": {
                 "大安": "表面财务稳定，但可能存在隐性问题。",
                 "留连": "财运进展缓慢，隐藏障碍可能影响收益。",
                 "速喜": "短期内有财务突破，但需防范隐性问题。",
                 "赤口": "财务冲突中可能存在隐患，需冷静处理。",
                 "小吉": "收益有所增加，但需警惕欺诈或隐性损失。",
                 "空亡": "财运不明朗，可能有隐性问题，建议审慎评估。"
            },
             "腾蛇": {
                 "大安": "财运看似稳定，但可能存在误导或虚假收益。",
                 "留连": "拖延中易受迷惑，需警惕虚假信息。",
                 "速喜": "短期收益可能较高，但需警惕投资陷阱。",
                 "赤口": "财务矛盾可能因虚假信息加剧，需核实清楚。",
                 "小吉": "收益有所增长，但需注意潜在虚假或误导。",
                 "空亡": "财运停滞不前，但可能有意外的灵感或机会。"
            }
        },
        6: {  # 身体状况
             "青龙": {
                 "大安": "健康状态较好，恢复较快，身体状况稳定。",
                 "留连": "小病缠身，但无大碍，需注意休息。",
                 "速喜": "恢复迅速，适合主动康复训练。",
                 "赤口": "健康欠佳，需特别注意炎症或意外伤害。",
                 "小吉": "健康小有改善，适合持续调养。",
                 "空亡": "健康状况较差，可能需紧急治疗。"
            },
             "朱雀": {
                 "大安": "喉咙、口腔易受影响，需注意饮食调节。",
                 "留连": "恢复较慢，可能因劳累导致疲惫。",
                 "速喜": "短期好转，但需注意急性炎症。",
                 "赤口": "易感染炎症，或因急躁引发健康问题。",
                 "小吉": "健康逐步恢复，但需注意情绪管理。",
                 "空亡": "健康隐患较多，需全面检查和及时干预。"
            },
             "勾陈": {
                 "大安": "身体稳定，但可能有慢性问题需调理。",
                 "留连": "恢复缓慢，可能存在反复。",
                 "速喜": "健康状况有所改善，但易受反复影响。",
                 "赤口": "病情可能因环境或压力加剧。",
                 "小吉": "身体恢复较慢，但逐步向好。",
                 "空亡": "健康状况停滞，可能需重新评估病情。"
            },
             "白虎": {
                 "大安": "身体状态平稳，但潜藏风险，需警惕意外伤害。",
                 "留连": "恢复延迟，可能因劳累或环境因素加剧问题。",
                 "速喜": "短期好转伴随风险，需避免过度活动。",
                 "赤口": "健康状况欠佳，易发生意外或加重病情。",
                 "小吉": "小有改善，但需避免疲劳。",
                 "空亡": "健康问题较严重，需防范重大疾病或突发状况。"
            },
             "玄武": {
                 "大安": "表面健康平稳，但可能隐藏问题需注意。",
                 "留连": "恢复缓慢，可能有隐性病灶。",
                 "速喜": "短期症状缓解，但需防复发。",
                 "赤口": "健康问题可能因隐性因素加剧。",
                 "小吉": "小有好转，但需注意隐患排查。",
                 "空亡": "健康状况复杂，需全面检查和专业治疗。"
            },
             "腾蛇": {
                 "大安": "健康表面稳定，但可能暗藏隐患。",
                 "留连": "病情可能反复，需长期关注。",
                 "速喜": "短期好转，但易复发或留下隐患。",
                 "赤口": "注意意外伤害或潜在问题加重。",
                 "小吉": "身体逐步恢复，但需防止复发或虚假好转。",
                 "空亡": "健康不稳定，可能需长时间治疗。"
            }
        }
        
    }

    # 获取对应问题类型的六神解释
    default_meaning = "无明确解释，需依据具体情况分析。"

    # 动态获取解释
    question_data = meanings_by_question.get(question_type, {})
    god_data = question_data.get(god, {})
    return god_data.get(palace, default_meaning)



In [55]:
def interpret_detailed_results(results, lunar_day, hour_branch, question_type):
    """
    小六壬详细解析：逐步解析六宫、六亲、六神、五星和其他符号。
    :param results: 小六壬的结果字符串。
    :param lunar_day: 农历日（用于三才格局）。
    :param hour_branch: 时支（用于三才格局）。
    :param question_type: 问题类型：
        1-运势，2-感情，3-工作，4-学业，5-占财，6-身体状况。
    :return: 针对每个宫位的详细解析。
    """
    # 问题类型映射
    question_types = {
        1: "运势",
        2: "感情",
        3: "工作",
        4: "学业",
        5: "占财",
        6: "身体状况"
    }
    question = question_types.get(question_type, "未知问题类型")

    # 六宫固定意义
    palace_meanings = {
        "大安": "事物平稳，稳定发展，适合积累和规划。",
        "留连": "事情拖延，反复难决，需耐心应对。",
        "速喜": "快速变化，机会较多，适合抓住时机。",
        "赤口": "冲突多，争执多，可能遇到不利局面。",
        "小吉": "小有收获，逐步推进，适合稳中求进。",
        "空亡": "迷茫不定，失去方向，宜谨慎行事。"
    }

    # 六神固定意义
    god_meanings = {
        "青龙": "吉利，象征好运与贵人相助。",
        "朱雀": "口舌是非，象征争执与沟通障碍。",
        "勾陈": "反复拖延，象征受阻与困境。",
        "白虎": "凶险，象征竞争与意外压力。",
        "玄武": "隐秘，象征欺骗与隐性问题。",
        "腾蛇": "迷惑，象征虚幻与难以捉摸。"
    }

    # 五行相生相克关系
    element_cycle = {
        "木": {"生": "水生木", "克": "木克土"},
        "火": {"生": "木生火", "克": "火克金"},
        "土": {"生": "火生土", "克": "土克水"},
        "金": {"生": "土生金", "克": "金克木"},
        "水": {"生": "金生水", "克": "水克火"}
    }

    # 五星动态意义（针对问题类型动态解释）
    def interpret_star_by_question(element, question_type):
        meanings = {
            1: {"木": "生机勃发，代表好运和新机会。", "火": "热情高涨，象征快速推进的运势。", "土": "稳定积累，代表运势逐步增强。", "金": "收获阶段，代表决断与成功。", "水": "灵活变化，适合调整策略。", "无": "不确定的运势，需要谨慎。"},
            2: {"木": "感情逐渐升温，适合新的开始。", "火": "感情热烈但可能有冲突。", "土": "感情稳定发展，但进展缓慢。", "金": "感情进入收获阶段，关系明确。", "水": "感情灵活多变，需调整沟通方式。", "无": "感情状态不明朗，需要观察。"},
            3: {"木": "新项目或机会，适合创新和开始。", "火": "快速推进的工作节奏，可能有压力。", "土": "工作稳定，适合积累和长期规划。", "金": "工作成果显现，适合收获阶段。", "水": "工作环境变化，需灵活应对。", "无": "工作方向不明，需要等待。"},
            4: {"木": "学习进展顺利，适合新课题的探索。", "火": "学习热情高，但可能容易分心。", "土": "学习基础扎实，适合复习和积累。", "金": "学习成果显现，适合收尾阶段。", "水": "学习思维灵活，适合拓展新领域。", "无": "学习状态不稳定，需要调整。"},
            5: {"木": "财源新生，适合开拓新机会。", "火": "财运快速提升，但需防范风险。", "土": "财运稳定积累，适合长期规划。", "金": "财务丰收，适合收尾阶段。", "水": "灵活生财，适合调整策略。", "无": "财运不明，需要观察。"},
            6: {"木": "健康逐步好转，适合康复与保养。", "火": "健康状况快速变化，需防止急症。", "土": "健康稳定，适合长期调理。", "金": "身体逐渐恢复，适合总结治疗效果。", "水": "健康状态波动，需适时调整。", "无": "健康状态不明，需要观察与检查。"}
        }
        return meanings.get(question_type, {}).get(element, "未知五星意义，需结合实际分析。")

    # 动态六亲意向
    def interpret_six_relative_by_question(relative, question_type):
        meanings = {
            1: {"父母": "支持和资源（外部助力）", "兄弟": "竞争与合作（环境影响）", "子孙": "成果与幸福（成功的可能性）", "妻财": "财富与资源（物质基础）", "官鬼": "阻力与困难（风险或挑战）", "自身": "当前状态（事情是否适合）"},
            2: {"父母": "家庭支持或外部压力", "兄弟": "情感中的第三方关系", "子孙": "感情的成果，婚姻或幸福指数", "妻财": "伴侣或感情中的物质支撑", "官鬼": "情感压力（是矛盾或困扰）", "自身": "自身状态（主观意愿）"},
            3: {"父母": "上级领导或支持性资源", "兄弟": "同事或竞争者的影响", "子孙": "工作成果或晋升机会", "妻财": "收入或物质回报", "官鬼": "压力或职场挑战", "自身": "自身能力和适配性"},
            4: {"父母": "师长帮助或学习资源", "兄弟": "竞争或同学的影响", "子孙": "学习成果或目标实现", "妻财": "学习投入的资源或支持", "官鬼": "学习难题或考试压力", "自身": "当前学习状态或适应性"},
            6: {"父母": "健康支持（治疗资源）", "兄弟": "健康中的隐性因素", "子孙": "健康的恢复与成果", "妻财": "健康的物质投入", "官鬼": "健康压力（病痛或困扰）", "自身": "当前健康状况"}
        }
        return meanings.get(question_type, {}).get(relative, "未知六亲意向，需结合实际分析。")



    # 综合解析
    if question_type == 5:
        for result in results.strip().split("\n"):
            if not result.strip():
                continue  # 跳过空行

            try:
                # 解析每一行结果
                parts = result.split()
                palace = parts[0].split("宫：")[0].strip()  # 宫位
                branch = parts[0].split("：")[1]  # 地支
                god = parts[1].split("：")[1]  # 六神
                element = parts[2].split("：")[1]  # 五星
                relative = parts[3].split("：")[1]  # 六亲
            except (IndexError, ValueError):
                continue

            if relative == "妻财":
                palace_desc = palace_meanings.get(palace, "未知宫位意义。")
                star_desc = interpret_star_by_question(element, question_type)
                god_meaning = god_meanings.get(god, "未知六神意义。")
                element_interpretation = element_cycle.get(element, {"生": "未知", "克": "未知"})
                god_summary = interpret_six_god_with_palace(palace, god,question_type)

                return (
                    f"妻财所在宫：{palace}\n"
                    f"{palace_desc}\n"
                    f"五星（{element}）：{star_desc}\n"
                    f"六神（{god}）：{god_meaning}\n"
                    f"总结：{god_summary}\n"
                )

        return "未找到妻财宫的信息，解析失败。"

    # 综合解析（非占财问题）
    interpretations = [f"问题类型：{question}"]

    for result in results.strip().split("\n"):
        if not result.strip():
            continue  # 跳过空行

        try:
            # 解析每一行结果
            parts = result.split()
            palace = parts[0].split("宫：")[0].strip()  # 宫位
            branch = parts[0].split("：")[1]  # 地支
            god = parts[1].split("：")[1]  # 六神
            element = parts[2].split("：")[1]  # 五星
            relative = parts[3].split("：")[1]  # 六亲
        except (IndexError, ValueError):
            interpretations.append(f"解析失败：{result}")
            continue

        # 获取动态意义
        relative_desc = interpret_six_relative_by_question(relative, question_type)
        palace_desc = palace_meanings.get(palace, "未知宫位意义。")
        star_desc = interpret_star_by_question(element, question_type)
        god_meaning = god_meanings.get(god, "未知六神意义。")
        god_summary = interpret_six_god_with_palace(palace, god,question_type)

        interpretations.append(
            f"在“{relative_desc}”方面：\n"
            f"{palace}宫: {palace_desc}\n"
            f"六神（{god}）：{god_meaning}\n"
            f"五星（{element}） ：{star_desc}\n"
            f"总结：{god_summary}\n"
        )

    interpretations.insert(0, f"整体分析：\n日干支（农历日）：{lunar_day}\n时支：{hour_branch}\n")

    return "\n".join(interpretations)





In [56]:
if __name__ == "__main__":
    # 输入公历年月日和时辰
    year = int(input("请输入年份（如2024）："))
    month = int(input("请输入月份（如11）："))
    day = int(input("请输入日期（如23）："))
    hour = int(input("请输入时辰（如14）："))
    print("请选择问题类型：")
    print("1 - 运势")
    print("2 - 感情")
    print("3 - 工作")
    print("4 - 学业")
    print("5 - 财运")
    print("6 - 身体状况")
    question_type = int(input("请输入问题类型对应的数字（如1）："))

    # 转换公历日期到农历日
    ganzhi_result = get_ganzhi(year, month, day, hour)
    lunar_day_index = ganzhi_result['lunar_day_index']
    day_ganzhi = ganzhi_result['day_ganzhi']
    hour_ganzhi = ganzhi_result['hour_ganzhi']
    hour_branch = ganzhi_result['hour_branch']
    print(f"日干支：{day_ganzhi}，时干支：{hour_ganzhi}")

    # 输出日时信息
    print(f"农历日：{lunar_day_index}，时支：{hour_branch}")

    # 调用小六壬生成结果
    raw_results = xiaoliuren_pan(lunar_day_index, hour_branch, None)
    print("四盘小六壬结果：")
    print(raw_results)

    # 调用解析函数
    #interpreted_results = interpret_xiaoliuren_results(raw_results, question_type)
    #print("\n解析结果：")
    #print(interpreted_results)

    print("\n详细解析结果：")
    interpreted_detailed_results = interpret_detailed_results(
        raw_results, lunar_day_index, hour_branch, question_type
    )
    print(interpreted_detailed_results)

请选择问题类型：
1 - 运势
2 - 感情
3 - 工作
4 - 学业
5 - 财运
6 - 身体状况
日干支：甲寅，时干支：甲戌
农历日：23，时支：戌
四盘小六壬结果：
大安宫：午 六神：勾陈 五星：土 六亲：父母
留连宫：申 六神：白虎 五星：金 六亲：子孙
速喜宫：戌 六神：玄武 五星：水 六亲：自身
赤口宫：子 六神：腾蛇 五星：无 六亲：妻财
小吉宫：寅 六神：青龙 五星：木 六亲：官鬼
空亡宫：辰 六神：朱雀 五星：火 六亲：兄弟

详细解析结果：
整体分析：
日干支（农历日）：23
时支：戌

问题类型：身体状况
在“健康支持（治疗资源）”方面：
大安宫: 事物平稳，稳定发展，适合积累和规划。
六神（勾陈）：反复拖延，象征受阻与困境。
五星（土） ：健康稳定，适合长期调理。
总结：身体稳定，但可能有慢性问题需调理。

在“健康的恢复与成果”方面：
留连宫: 事情拖延，反复难决，需耐心应对。
六神（白虎）：凶险，象征竞争与意外压力。
五星（金） ：身体逐渐恢复，适合总结治疗效果。
总结：恢复延迟，可能因劳累或环境因素加剧问题。

在“当前健康状况”方面：
速喜宫: 快速变化，机会较多，适合抓住时机。
六神（玄武）：隐秘，象征欺骗与隐性问题。
五星（水） ：健康状态波动，需适时调整。
总结：短期症状缓解，但需防复发。

在“健康的物质投入”方面：
赤口宫: 冲突多，争执多，可能遇到不利局面。
六神（腾蛇）：迷惑，象征虚幻与难以捉摸。
五星（无） ：健康状态不明，需要观察与检查。
总结：注意意外伤害或潜在问题加重。

在“健康压力（病痛或困扰）”方面：
小吉宫: 小有收获，逐步推进，适合稳中求进。
六神（青龙）：吉利，象征好运与贵人相助。
五星（木） ：健康逐步好转，适合康复与保养。
总结：健康小有改善，适合持续调养。

在“健康中的隐性因素”方面：
空亡宫: 迷茫不定，失去方向，宜谨慎行事。
六神（朱雀）：口舌是非，象征争执与沟通障碍。
五星（火） ：健康状况快速变化，需防止急症。
总结：健康隐患较多，需全面检查和及时干预。



In [43]:
def interpret_find_target_detailed(results, option, lunar_day_index):
    """
    根据输入选项详细解析寻人或寻物的目标宫位和结果。
    :param results: 小六壬结果字符串。
    :param option: 寻人/寻物类型，数字表示：
        1 - 寻找妻子（妻财宫）
        2 - 寻找丈夫或官员（官鬼宫）
        3 - 寻找陌生人（日宫）
        4 - 寻找文书（父母宫）
        5 - 寻找宠物（子孙宫）
        6 - 寻找财务（妻财宫）
    :param lunar_day_index: 农历日索引（1-30）。
    :return: 针对选项的解析结果。
    """
    # 宫位规则
    palace_rules = {
        "大安": {
            "人": "还没走远。",
            "物": "在家或附近，可能藏在显眼处。"
        },
        "留连": {
            "人": "不能归。",
            "物": "在附近，没离开原地，被东西盖住了。"
        },
        "速喜": {
            "人": "快回来了。",
            "物": "在人多喧哗的地方，易被找到。"
        },
        "赤口": {
            "人": "有事绊住了。",
            "物": "东西在角落或隐蔽处，需仔细寻找。"
        },
        "小吉": {
            "人": "在回来的路上。",
            "物": "丢在路边或在路上丢的。"
        },
        "空亡": {
            "人": "没定归期。",
            "物": "很难找到，可能彻底丢失。"
        }
    }

    # 六宫对应方向
    palace_directions = {
        "大安": "东",
        "留连": "东南",
        "速喜": "南",
        "赤口": "西南",
        "小吉": "西北",
        "空亡": "北或东北"
    }

    # 六神对目标的影响
    god_meanings_target = {
        "青龙": "可能有贵人相助，目标容易找到。",
        "朱雀": "可能需要通过沟通或询问才能找到。",
        "勾陈": "目标可能被阻碍或隐藏，需耐心寻找。",
        "白虎": "目标可能遇到损坏或冲突。",
        "玄武": "目标可能涉及隐秘或欺骗，找回较难。",
        "腾蛇": "目标可能因复杂情况暂时难以找到。"
    }

    # 动态获取日宫位置
    def get_day_palace(lunar_day_index):
        six_positions = ["大安", "留连", "速喜", "赤口", "小吉", "空亡"]
        return six_positions[(lunar_day_index - 1) % 6]

    # 特殊处理日宫
    if option == 3:
        day_palace = get_day_palace(lunar_day_index)
    else:
        day_palace = None

    # 对应目标六亲
    target_mapping = {
        1: "妻财",  # 寻找妻子
        2: "官鬼",  # 寻找丈夫或官员
        3: "日宫",  # 寻找陌生人
        4: "父母",  # 寻找文书
        5: "子孙",  # 寻找宠物
        6: "妻财"   # 寻找财务
    }

    target_relative = target_mapping.get(option, None)
    if not target_relative:
        return "无效的输入选项，请输入正确的选项数字（1-6）。"

    # 综合解析
    target_results = [f"寻找目标解析结果："]

    found = False
    for result in results.strip().split("\n"):
        if not result.strip():
            continue

        try:
            # 解析每一行结果
            parts = result.split()
            palace = parts[0].split("宫：")[0].strip()  # 宫位
            god = parts[1].split("：")[1]  # 六神
            relative = parts[3].split("：")[1]  # 六亲
        except (IndexError, ValueError):
            target_results.append(f"解析失败：{result}")
            continue

        # 如果是日宫，匹配日宫对应的宫位
        if option == 3 and palace == day_palace:
            found = True
            direction = palace_directions.get(palace, "未知方向")
            person_status = palace_rules[palace]["人"]
            god_meaning = god_meanings_target.get(god, "未知六神影响，需结合实际分析。")
            target_results.append(
                f"{palace}宫（日宫）：方位为{direction}。\n"
                f"状态：{person_status}\n"
                f"六神影响：{god_meaning}\n"
            )
        # 常规目标六亲匹配
        elif relative == target_relative:
            found = True
            direction = palace_directions.get(palace, "未知方向")
            item_status = palace_rules[palace]["物"] if "物" in palace_rules[palace] else "未知物品状态"
            god_meaning = god_meanings_target.get(god, "未知六神影响，需结合实际分析。")
            target_results.append(
                f"{palace}宫：方位为{direction}。\n"
                f"状态：{item_status}\n"
                f"六神影响：{god_meaning}\n"
            )

    if not found:
        target_results.append("未找到目标相关信息，可能需要更详细的排盘结果。")

    return "\n".join(target_results)

In [44]:
if __name__ == "__main__":
    # 输入公历年月日和时辰
    year = int(input("请输入年份（如2024）："))
    month = int(input("请输入月份（如11）："))
    day = int(input("请输入日期（如23）："))
    hour = int(input("请输入时辰（如14）："))
    
    print("请选择目标类型：")
    print("1 - 寻找妻子")
    print("2 - 寻找丈夫或官员")
    print("3 - 寻找陌生人")
    print("4 - 寻找文书")
    print("5 - 寻找宠物")
    print("6 - 寻找财务")
    option = int(input("请输入对应数字（1-6）："))

    # 转换公历日期到农历日
    try:
        ganzhi_result = get_ganzhi(year, month, day, hour)
        lunar_day_index = ganzhi_result['lunar_day_index'] + 1  # 农历日从1开始计
        day_ganzhi = ganzhi_result['day_ganzhi']
        hour_ganzhi = ganzhi_result['hour_ganzhi']
        hour_branch = ganzhi_result['hour_ganzhi'][-1]  # 取时支
    except Exception as e:
        print(f"日期转换错误：{e}")
        exit(1)

    # 打印干支信息
    print(f"日干支：{day_ganzhi}，时干支：{hour_ganzhi}")
    print(f"农历日：{lunar_day_index}，时支：{hour_branch}")


    # 调用寻人寻物解析函数
    try:
        target_analysis = interpret_find_target_detailed(results, option, lunar_day_index)
        print(target_analysis)
    except Exception as e:
        print(f"目标解析错误：{e}")
        exit(1)


请选择目标类型：
1 - 寻找妻子
2 - 寻找丈夫或官员
3 - 寻找陌生人
4 - 寻找文书
5 - 寻找宠物
6 - 寻找财务
日干支：乙卯，时干支：丁丑
农历日：14，时支：丑
寻找目标解析结果：
留连宫（日宫）：方位为东南。
状态：不能归。
六神影响：可能有贵人相助，目标容易找到。

